<a href="https://colab.research.google.com/github/lapatradaa/M-MMT4NL/blob/main/llms_evaluation_direct_translate_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install sacrebleu bert-score
!pip install sacrebleu bert-score pythainlp
!pip install textdistance

!pip -q install pandas openpyxl jellyfish bert-score tqdm openai --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 88.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 355.9/355.9 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 948.6/948.6 kB 37.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.3.2 which is incompatible.
dask-cudf-cu12 25.6.0 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.2 which is incom

In [5]:
import os, random, math
from pathlib import Path
import pandas as pd
import random
import os
from tqdm import tqdm
import jellyfish
from bert_score import score as bertscore


In [6]:
OPENAI_MODEL_TRANSLATE = "gpt-4o-mini"
OPENAI_MODEL_PERTURB  = "gpt-4o-mini"
TEMPERATURE_TRANSLATE  = 0.2
TEMPERATURE_PERTURB    = 0.7


In [7]:
# multilingual BERTScore model (works well for Thai)
BERT_MODEL = "xlm-roberta-large"
W_BERT = 0.8
W_JARO = 0.2

RANDOM_SEED = 42
random.seed(RANDOM_SEED)

In [19]:
from openai import OpenAI

##_client = OpenAI <“OPENAI_API_KEY”>

def _chat_once(system_prompt: str, user_prompt: str, *, model: str, temperature: float) -> str:
    out = _client.chat.completions.create(
        model=model,
        temperature=temperature,
        messages=[
            {"role": "system", "content": system_prompt.strip()},
            {"role": "user",   "content": user_prompt.strip()},
        ],
    )
    return (out.choices[0].message.content or "").strip()

In [9]:
# --- Prompt Templates ---
PROMPT_TAXONOMY = """You're an expert linguist in English and Thai. You need to modify this Thai sentence by substituting a word with its respective synonym, while still keeping the whole semantic of the sentence.

Examples of this modifications in English are as follows.

I'll give you some examples of converting one sentence to another sentence: "I'm so tired" is converted to "I'm so exhausted" "I'm really hungry" is converted to "I'm really starving" "I'm not sure if I'm up for that" is converted to "I'm not certain if I'm up for that" "I'm not sure if I can make it to the event" is converted to "I'm not confident if I can make it to the event"

Can you apply this concept to the Thai sentence below. Only show the modified sentence without any explanation.
You're an expert linguist in English and Thai. You need to modify this Thai sentence by negating the sentence, while still keeping the whole semantic of the sentence.

Examples of this modifications in English are as follows.

"I'm so tired" is converted to "I'm so not energetic"
"I'm really hungry" is converted to "I'm really not full"
"I'm not sure if I'm up for that" is converted to "I'm sure I'm not up for that"
"I'm not sure if I can make it to the event" is converted to "I'm unsure if I can make it to the event"
"I'm feeling a bit confused right now" is converted to "I'm feeling a bit not clear right now"

Can you apply this concept to the Thai sentence below. Only show the modified sentence without any explanation.

{thai_text}

"""


In [10]:
PROMPT_NER = """Can you apply this concept to the Thai sentence below. Only show the modified sentence without any explanation.
You're an expert linguist in English and Thai. You need to modify this Thai sentence by replacing the named entity with a new name, while still keeping the whole semantic of the sentence.

Examples of this modifications in English are as follows.

"I'm so tired" is converted to "Jane is so tired"
"I'm really hungry" is converted to "Jack is really hungry" "I'm not sure if I'm up for that" is converted to "Jones is not sure if she is up for that"
"I'm not sure if I can make it to the event" is converted to "Jill is not sure if she can make it to the event"
"I'm feeling a bit confused right now" is converted to "Andy is feeling a bit confused right now"

Can you apply this concept to the Thai sentence below. Only show the modified sentence without any explanation.

{thai_text}
"""

In [11]:
PROMPT_NEGATION = """Can you apply this concept to the Thai sentence below. Only show the modified sentence without any explanation.

You're an expert linguist in English and Thai. You need to modify this Thai sentence by negating the sentence, while still keeping the whole semantic of the sentence.

Examples of this modifications in English are as follows.

"I'm so tired" is converted to "I'm so not energetic"
"I'm really hungry" is converted to "I'm really not full"
"I'm not sure if I'm up for that" is converted to "I'm sure I'm not up for that"
"I'm not sure if I can make it to the event" is converted to "I'm unsure if I can make it to the event"
"I'm feeling a bit confused right now" is converted to "I'm feeling a bit not clear right now"

Can you apply this concept to the Thai sentence below. Only show the modified sentence without any explanation.



{thai_text}
"""

In [12]:
def perturb_thai(thai_text: str, taxonomy: str):
    if taxonomy == "taxonomy":
        prompt = PROMPT_TAXONOMY.format(thai_text=thai_text)
    elif taxonomy == "ner":
        prompt = PROMPT_NER.format(thai_text=thai_text)
    else:
        raise ValueError("taxonomy must be taxonomy / ner")

    resp = _client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role":"system","content":"You are a helpful assistant."},
                  {"role":"user","content":prompt}],
        temperature=0.7
    )
    return resp.choices[0].message.content.strip()

In [13]:
import textdistance

def jaro_score(ref: str, cand: str) -> float:
    """
    Compute Jaro-Winkler similarity between reference and candidate.
    Value between 0 (completely different) and 1 (exact match).
    """
    if not ref.strip() or not cand.strip():
        return 0.0
    return textdistance.jaro_winkler(ref, cand)


In [14]:
def bert_f1(ref: str, cand: str) -> float:
    if not ref.strip() or not cand.strip():
        return 0.0
    try:
        P, R, F1 = bertscore(
            cands=[cand],
            refs=[ref],
            model_type="xlm-roberta-large",
            lang="th",
            rescale_with_baseline=False,  # non-baseline for th
            verbose=False
        )
        return float(F1[0].item())
    except Exception:
        return 0.0

In [15]:
def fitness(ref_thai: str, cand_thai: str, W_BERT=0.8, W_JARO=0.2):
    b = bert_f1(ref_thai, cand_thai)
    j = jaro(ref_thai, cand_thai)
    f = W_BERT * b + W_JARO * j
    return round(f, 4), round(b, 4), round(j, 4)

In [16]:
def evaluate_sentence(sentence: str, rounds: int = 3, perturb_type: str = "taxonomy", weight_bert: float = 0.7):
    """
    Evaluate a sentence with perturbations and compute a fitness score per round.
    Returns all rounds plus highlights the best round.
    """
    results = []
    best_score = -1
    best_round = None

    for i in range(1, rounds + 1):

        # --- Select prompt ---
        if perturb_type == "taxonomy":
            prompt = PROMPT_TAXONOMY.format(thai_text=sentence)
        elif perturb_type == "ner":
            prompt = PROMPT_NER.format(thai_text=sentence)
        elif perturb_type == "negation":
            prompt = PROMPT_NEGATION.format(thai_text=sentence)
        else:
            raise ValueError("Invalid perturb_type. Choose 'taxonomy', 'ner', or 'negation'.")

        # --- Perturb sentence ---
        perturbed = _chat_once(
            system_prompt="You are a helpful Thai linguist.",
            user_prompt=prompt,
            model="gpt-4o-mini",
            temperature=0.7
        )

        # --- Scores (directly compare text) ---
        bert_sim = bert_f1(sentence, perturbed)
        jaro_sim = jaro_score(sentence, perturbed)
        fitness = weight_bert * bert_sim + (1 - weight_bert) * jaro_sim

        # Track best round
        if fitness > best_score:
            best_score = fitness
            best_round = i

        results.append({
            "round": i,
            "original": sentence,
            "perturb_type": perturb_type,
            "perturbed": perturbed,
            "bert_f1": bert_sim,
            "jaro": jaro_sim,
            "fitness": fitness
        })

    # Mark the best round
    for r in results:
        r["best_round"] = (r["round"] == best_round)

    return results


In [17]:
def run_file(input_path: str, output_path: str, perturb_type: str = "taxonomy", rounds: int = 3):
    """
    Run perturbation evaluation on all sentences in an Excel file.
    Handles both 'sentence' and 'original' column names.
    Saves results into a new Excel file.
    """
    # --- Load input file ---
    df = pd.read_excel(input_path)

    # Detect column name
    if "sentence" in df.columns:
        col_name = "sentence"
    elif "original" in df.columns:
        col_name = "original"
    else:
        raise ValueError("Excel must have a column named 'sentence' or 'original'.")

    all_results = []

    # --- Loop over rows ---
    for idx, row in tqdm(df.iterrows(), total=len(df), desc=f"Evaluating {perturb_type}"):
        sentence = str(row[col_name])
        results = evaluate_sentence(sentence, rounds=rounds, perturb_type=perturb_type)

        # Flatten results for Excel
        for r in results:
            all_results.append({
                "index": idx,
                "round": r["round"],
                "original": r["original"],
                "perturb_type": r["perturb_type"],
                "perturbed": r["perturbed"],
                "bert_f1": r["bert_f1"],
                "jaro": r["jaro"],
                "fitness": r["fitness"],
                "best_round": r["best_round"]
            })

 # --- Save output ---
    out_df = pd.DataFrame(all_results)
    out_df.to_excel(output_path, index=False)
    print(f"✅ Saved results to {output_path}")

In [15]:
run_file("/content/Test2_mmmt4nl (2).xlsx", "output_taxonomy.xlsx", perturb_type="taxonomy", rounds=5)
run_file("/content/Test2_mmmt4nl (2).xlsx", "output_ner.xlsx", perturb_type="ner", rounds=5)


Evaluating taxonomy:   0%|          | 0/50 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

Evaluating taxonomy: 100%|██████████| 50/50 [20:11<00:00, 24.24s/it]


✅ Saved results to output_taxonomy.xlsx


Evaluating ner: 100%|██████████| 50/50 [19:00<00:00, 22.81s/it]

✅ Saved results to output_ner.xlsx


In [18]:
run_file("/content/Test2_mmmt4nl (2).xlsx", "output_negation.xlsx", perturb_type="negation", rounds=5)


Evaluating negation:   0%|          | 0/50 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

Evaluating negation: 100%|██████████| 50/50 [18:51<00:00, 22.63s/it]

✅ Saved results to output_negation.xlsx
